In [4]:
from bertopic import BERTopic
from hdbscan import HDBSCAN
from bertopic.representation import KeyBERTInspired
from bertopic.representation import OpenAI
import openai
import httpx
from bertopic.representation import MaximalMarginalRelevance
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
# from nltk.corpus import stopwords
# from bertopic.backend import OpenAIBackend

In [5]:
category = "TOOLS"
df = pd.read_csv('data/%s_apps.csv' % category)
docs = list(df['description'])

In [6]:
hdbscan_model = HDBSCAN(min_cluster_size=100, prediction_data=True)

# standard_stopwords = list(stopwords.words('english'))
# additional_stopwords = ['app', 'application', 'mobile']
# full_stopwords = standard_stopwords + additional_stopwords
# vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words='english', min_df=10)  # stop_words=full_stopwords

# vectorizer_model = CountVectorizer(stop_words='english')


In [ ]:
client = openai.OpenAI(
    base_url = "https://api.deepseek.com",
    api_key = "sk-61de429c43324f1e90a41e8610637a5e",
    # base_url="https://api.xty.app/v1",
    # api_key="sk-Y2rUKBk6to7SoIBjBaC72894Dc8e45B8A5E43f57Be37B67a", 
    # http_client=httpx.Client(
    #     base_url="https://api.xty.app/v1",
    #     follow_redirects=True,
    # ),
)
representation_model_ins = KeyBERTInspired()
representation_model_gpt = OpenAI(client, model="gpt-4o", chat=True)
representation_model_insmmr = [KeyBERTInspired(top_n_words=20),
                               MaximalMarginalRelevance(diversity=.5)]

# embedding_model = OpenAIBackend(client, "text-embedding-3-small")

representation_models = {
    "Main": representation_model_ins,
    "Aspect1": representation_model_insmmr,
    "Aspect2": representation_model_gpt
}

topic_model = BERTopic(nr_topics='auto', n_gram_range=(1, 3), min_topic_size=100,
                       calculate_probabilities=True,
                       hdbscan_model=hdbscan_model,
                       # vectorizer_model=vectorizer_model,
                       # embedding_model=embedding_model,
                       representation_model=representation_models
                      )

# topic_model.save('my_bt_model')
# topic_model.representation_model=representation_models

topics, probs = topic_model.fit_transform(docs)

# standard_stopwords = list(stopwords.words('english'))
# additional_stopwords = ['app', 'application', 'mobile']
# full_stopwords = standard_stopwords + additional_stopwords
vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words='english', min_df=0.01, max_df=0.99)

topic_model.update_topics(docs, vectorizer_model=vectorizer_model)

print(topic_model.get_topic_info())

topic_model.get_topic_info().to_csv('cluster/%s-topic-2.csv' % category)

topic_model.visualize_topics()

topic_model.visualize_heatmap()


In [ ]:

topics_to_merge = [0, 1, 6, 8]
topic_model.merge_topics(docs, topics_to_merge)

# Access updated topics
topics = topic_model.topics_

topic_model.get_topic_info().to_csv('cluster/%s-topic-1-merged.csv' % category)
topic_model.get_document_info(docs).to_csv('cluster/BEAUTY-document.csv')
topic_model.visualize_topics()
topic_model.visualize_heatmap()

In [64]:

# # Reduce outliers
# new_topics = topic_model.reduce_outliers(docs, topics)

# # Reduce outliers using the `probabilities` strategy
# new_topics = topic_model.reduce_outliers(docs, topics, probabilities=probs, strategy="probabilities")

# # Reduce outliers using the `distributions` strategy
# new_topics = topic_model.reduce_outliers(docs, topics, strategy="distributions")

# # Reduce outliers using the `c-tf-idf` strategy
# new_topics = topic_model.reduce_outliers(docs, topics, strategy="c-tf-idf")

# Reduce outliers using the `embeddings` strategy
new_topics = topic_model.reduce_outliers(docs, topics, strategy="embeddings")

# 创建数据框
df = pd.DataFrame({"Document": docs, "Assigned Topic": new_topics})
df.to_csv('cluster/%s-document-xxx.csv' % category)
